# Генерація тексту

У цій роботі ми використаємо мовну модель для генерації тексту.

У класичної мовної моделі є два взаємопов'язані визначення:

1. Оцінити ймовірність вхідного тексту.
2. Видати ймовірнісний розподіл наступного слова для даного префіксу.

Для генерації тексту нам ідеально підходить друге визначення.

## Початок роботи

Будь ласка, заповніть поля `EMAIL`, `NAME` та `GROUP` нижче:

In [1]:
!pip install --quiet --ignore-installed http://nlp.band/static/pypy/lpnlp-2023.10.2-py3-none-any.whl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.3/167.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 4.5 MB/s eta 0:00:00


In [2]:
################################################################################
# FILL-IN:
#-----------------------------------------------------------------------
EMAIL = "vasyl.rusyn.kn.2021@lpnu.ua"  # заповніть вашим значенням
################################################################################

import lpnlp

lab = lpnlp.start(email=EMAIL, lab="text_generation")

Удачі!


### Завантаження моделі

Тренування мовної моделі з нуля займає багато часу: від кількох годин або днів для маленьких та середніх моделей й аж до кількох місяців чи навіть років для великих. Звичайно, за рахунок розпаралелювання тренування на багатьох GPU, реальний час рідко буває більше місяця-двох.

Для цієї роботи (як і в реальному житті), ми візьмемо претреновану модель. Хтось натренував її за нас. В данному випадку, візьмемо GPT-2 від компанії OpenAI.

Ця модель, як і безліч інших, зберігається на [HuggingFace Models](https://huggingface.co/models). Завантажити та працювати з нею зручно через бібліотеку [HuggingFace Transformers](https://github.com/huggingface/transformers/)

In [3]:
!pip install transformers

In [4]:
import transformers
import torch

In [5]:
model = transformers.AutoModelForCausalLM.from_pretrained("gpt2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Токенізація

Кожна модель має словник токенів, з яким вона тренувалася, та правила токенізації (розбиття тексту на токени). Обов'язково слід використовувати той самий словник та метод.

Бібліотека transformers вміє робити це для кожної підтримуваної моделі.

In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Перевіримо токенізатор:

In [7]:
input_text = "Hello, LP NLP!"
token_ids = tokenizer.encode(input_text)
token_ids

[15496, 11, 18470, 399, 19930, 0]

In [8]:
tokenizer.convert_ids_to_tokens(token_ids)

['Hello', ',', 'ĠLP', 'ĠN', 'LP', '!']

Рідкісні слова будуть розбиті на підслова:

In [9]:
tokenizer.convert_ids_to_tokens(tokenizer.encode("My name is Oleksiy Syvokon"))

['My', 'Ġname', 'Ġis', 'ĠOle', 'ks', 'iy', 'ĠSy', 'v', 'ok', 'on']

## Перевірка моделі -- один крок ітерації

Ми генеруватимемо текст в циклі токен за токеном, зліва направо. Але для початку розберімо, як виглядає один крок такого циклу.


На кожному кроці на вхід моделі подаємо ті токени, які вже згенеровано.

На першому кроці у нас ще нічого не згенеровано. Тому починаємо зі токену `<BOS>` ("begin of sentence"). В різних моделях він виглядає по-різному. Подивимося, що в GPT-2:


In [10]:
tokenizer.bos_token

'<|endoftext|>'

In [11]:
tokenizer.bos_token_id

50256

In [12]:
# Формуємо вхідний батч. Майже завжди для більшої ефективності
# нейронні мережі очікують на вхід кілька незалежних речень
# (або зображень у випадку з комп'ютерним зором)
#
# В нашій роботі ми завжди працюємо лише з одним реченням,
# тож розмір батча дорівнює одинці. Але все одно маємо
# оформити вхід як матрицю:
input_ = torch.LongTensor([[tokenizer.bos_token_id]])
input_

tensor([[50256]])

In [13]:
# Нарешті робимо крок генерації
output = model(input_, return_dict=True)

# На виході модель повертає вектор з logits -- ненормалізованими
# ймовірностями кожного токена в словнику
output.logits.shape

torch.Size([1, 1, 50257])

Чому маємо саме таку розмірність?

* Перша одиниця -- розмір батча, тобто кількість вхідних речень.
* Друга одиниця -- це довжина вхідної послідовності. На першому кроці ми подали лише один токен (bos_token)
* 50257 -- це розмір словника

In [14]:
tokenizer.vocab_size

50257

Поки що модель повернула не ймовірності, а просто якісь числа. Їм треба нормалізувати функцією softmax:

In [15]:
probs = torch.softmax(output.logits, dim=-1)

In [16]:
# Розмірність має збігатися з розміром словника
probs.shape

torch.Size([1, 1, 50257])

In [17]:
# Сума ймовірностей має дорівнювати 1.0
probs.sum()

tensor(1.0000, grad_fn=<SumBackward0>)

Ймовірності можуть ставати дуже маленькими та причиняти проблеми у зв'язку з обмеженою точністю float чисел. Тому прийнято працювати з логарифмами ймовірностей.

In [18]:
log_probs = torch.log_softmax(output.logits, dim=-1)

# Щоб перейти до звичайних ймовірностей, маємо зробити експоненціювання
log_probs.exp().sum()

tensor(1., grad_fn=<SumBackward0>)

Кожному слову в словнику відповідає своя ймовірність бути побаченим
після заданого префікса. Префіксом у нас поки що був лише одни `bos_token`.


In [19]:
# Яка ймовірність, що речення почнеться зі слів "I", "red", "Why"?
for word in ("I", "red", "Why"):
    index = tokenizer.convert_tokens_to_ids(word)
    prob = log_probs[0, 0, index].exp()
    print(f"P({word}) = {prob}")

P(I) = 0.01832086220383644
P(red) = 0.00010652346827555448
P(Why) = 0.0008686604560352862


Тепер ми маємо ймовірностний розподіл по словнику. Можемо обрати слово, яке вважатимемо згенерованим. Тут можливі кілька стратегій, які ми розглянемо в наступних розділах.

## Greedy decoding

Найпростіший (але й не дуже цікавий) спосіб -- це завжди обирати токен з найбільшою ймовірністю:

In [20]:
next_token_id = log_probs.argmax()
next_token_id

tensor(198)

In [21]:
probs[-1, -1, 198]

tensor(0.0623, grad_fn=<SelectBackward0>)

In [22]:
tokenizer.convert_ids_to_tokens([next_token_id])  # Наш перший згенерований токен

['Ċ']

Зберемо код докупи та додамо цикл. В циклі ми продовжуватимемо генерувати текст токен за токеном, поки не настане одна з двох умов:
1. Модель видала спецальний токен `eos_token` (end of sentence)
2. Довжина згенерованого тексту перевищила певний поріг `max_len`

У хорошої моделі в більшості випадків має спрацьовувати перша умова зупинки. Проте іноді модель може впасти в безкінчений цикл. Щоб цьому запобігти, маємо другу умову.

In [33]:
@torch.no_grad()
def greedy_decode(model, tokenizer, max_len=50):
    start_index = tokenizer.bos_token_id
    result = [start_index]

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_, return_dict=True)
        log_probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = log_probs.argmax()
        # new_token = tokenizer.convert_ids_to_tokens(token_index)

        # Зупиняємося, якщо досягли кінця
        if token_index == tokenizer.eos_token_id:
          break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    return result


generated_token_ids = greedy_decode(model, tokenizer)

Маємо список згенерованих індексів токенів, який починається ось так:

In [34]:
generated_token_ids[:10]

[50256, 198, 464, 717, 640, 314, 2497, 262, 649, 2196]

Перетворимо їх в текст:

In [35]:
tokens = tokenizer.convert_ids_to_tokens(generated_token_ids)
tokens[:10]

['<|endoftext|>',
 'Ċ',
 'The',
 'Ġfirst',
 'Ġtime',
 'ĠI',
 'Ġsaw',
 'Ġthe',
 'Ġnew',
 'Ġversion']

In [36]:
generated_text = "".join(tokens)
generated_text

'<|endoftext|>ĊTheĠfirstĠtimeĠIĠsawĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcited.ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠto'

In [37]:
lab.checkpoint("greedy decode", generated_text)

Відповідь правильна ✅



'<|endoftext|>ĊTheĠfirstĠtimeĠIĠsawĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcited.ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠtoĠseeĠtheĠnewĠversionĠofĠtheĠgame,ĠIĠwasĠsoĠexcitedĠto'

### Примітка: Byte-pair encoding (BPE)

Наша модель використовує subword токенізацію, а саме byte-pair encoding (BPE). В сучасному NLP це найрозповсюдженіший спосіб токенізації. Детально можете подивитися в [цьому відео](https://www.youtube.com/watch?v=tOMjTCO0htA).

Для наших цілей зараз важливо, що BPE заміняє пробіли на спеціальні Unicode-символи "Ġ". Серед інших моделей широко поширений варіант "▁" (зверніть увагу, це не звичайний символ підкреслення "_"). Щоб отримати чистий текст, треба виконати наступну заміну:

In [38]:
def bpe_decode(s):
    result = s.replace("Ġ", " ")
    result = result.replace("Ċ", "\n")
    return result

bpe_decode(generated_text)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

Але краще довіритися токенізатору й зробити цю роботу за нас:

In [39]:
tokenizer.convert_tokens_to_string(tokens)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

## Generic decoding function

Обирати слово з найбільшою ймовірністю -- не найкращий варіант для генерації тексту хоча б тому, що він завжди детерміновано призводить до однієї послідовності. Нижче ми подивимося на цікавіші альтернативи.

Цикл генерації залишиться той самий, що і в `greedy_decode()`. Відрізнятися буде лише один рядок -- той, в якому ми приймали рішення, яке слово обрати. Для зручності, винесемо цей рядок в окрему функцію. Ця функція прийматиме на вхід ймовірностний розподіл по словнику і повертає обраний токен.

In [40]:
def greedy_choice(probs):
    return probs.argmax()

Функцію генерації також трохи переробимо.

По-перше, додамо параметр `sample_fn` -- це має бути функція, яка обирає слово з ймовірностного розподілу, наприклад, `greedy_choice`.

По-друге, для зручності виконуватимемо BPE декодинг у середині функції генерації.

In [46]:
@torch.no_grad()
def generate(model, tokenizer, sample_fn, max_len=50, bpe_decode=True):
    start_index = tokenizer.bos_token_id
    result = [start_index]

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_, return_dict=True)
        log_probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = sample_fn(log_probs.exp())     # <---------------- цей рядок змінено

        # Зупиняємося, якщо досягнули кінця
        if token_index == tokenizer.eos_token_id:
          break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    if bpe_decode:
        tokens = tokenizer.convert_ids_to_tokens(result)
        result = tokenizer.convert_tokens_to_string(tokens)

    return result


generate(model, tokenizer, greedy_choice)

'<|endoftext|>\nThe first time I saw the new version of the game, I was so excited. I was so excited to see the new version of the game, I was so excited to see the new version of the game, I was so excited to'

## Simple sampling

Перший альтернатива -- це sampling. Тепер ми обираємо наступний токен випадково, але пропорційно до ймовірностей.

In [47]:
def simple_sample(probs):
    return torch.multinomial(probs, num_samples=1)[0]

# Згенеруємо 5 речень
for i in range(1, 6):
    result = generate(model, tokenizer, simple_sample)
    print(f"#{i}: {result}")
    print()

#1: <|endoftext|>We've been waiting for a sweet, juicy and nutritious combo meal taste great with an inexpensive platter of sweet and hot seafood and eggs and eggs in advance of dinner, but I'm hesitant to try so much flake and then taste the d

#2: <|endoftext|>Ooope Wild, Chief Keef, and James Spader did it blindfolded and then walked 24 yards in a 17-1 split. Wizards players learned to cover the ball so well against the run with a totally unperturbed Sh

#3: <|endoftext|>For new readers, I took the time to examine some DNA studies – with regards to the Q403 gene — and one interesting study: one with 23 Jensen models to examine the relationship between distance between observed and unrecorded bisexuality. (According to

#4: <|endoftext|>A recent climate study found that phosphoric acid (PPA) directly regulated the ability of squid to adapt to rising sea levels. Cried about by researcher Mary Betuskas over at Climategate, the researchers tested the water supply to Sixty

#5: <|endofte

## Sampling with temperature

Ми також можемо впливати на генерацію параметром температури softmax.

Більші значення температури призводять до того, що різниця між ймовірностями токенів зменшується, тобто розподіл стає більш рівномірним. На практиці це означає, що менш ймовірні варіанти обиратимуться частіше і згенерований текст може бути цікавішим. Однак якщо продовжувати піднімати температуру, то текст спочатку втратить зв'язність, далі почнуть розпадатися слова та граматичність.

Менші значення температури змінюють розподіл таким чином, що основна ймовірніста маса припадає на невелику кількість топових токенів. При температурі 0 вся ймовірність дістанеться одному токену й семплінг перетвориться на greedy decoding.

Згенеруємо тексти з різною температурою:

In [48]:
for temperature in (0.1, 0.3, 0.5, 0.8, 1.0, 1.25, 1.5, 2.0, 3.0, 5.0):

    def sample_with_temp(probs):
        updated_probs = probs.log().div(temperature).exp()
        return simple_sample(updated_probs)

    print(f"Sampling with temperature={temperature}")
    result = generate(model, tokenizer, sample_with_temp, max_len=15)
    print(result)
    print()

Sampling with temperature=0.1
<|endoftext|>
The first time I saw the new version of the game, I

Sampling with temperature=0.3
<|endoftext|>
"I'm not sure that I would have done it if I

Sampling with temperature=0.5
<|endoftext|>I have used the following brands in the past:

Laser

Sampling with temperature=0.8
<|endoftext|>, which is closely based on sales of at least 4,000 TV

Sampling with temperature=1.0
<|endoftext|>merchet vanka vaghdik karvag events v

Sampling with temperature=1.25
<|endoftext|>Like

Looking at some "tree shaking NVIDIA house Linux rider what

Sampling with temperature=1.5
<|endoftext|>ethyl hydrodict=8988676 947332270 Dominant m

Sampling with temperature=2.0
<|endoftext|>ote庸, initially exclaim Innovation allexp Ire mailmail relinders

Sampling with temperature=3.0
<|endoftext|>Delivery alarm bottle sucking airstrike munitions Z tracking prime mash protocol laws lun platform

Sampling with temperature=5.0
<|endoftext|> STD breed studied Mak jun EE 308 Veteri

In [49]:
# Яке значення `temperature` здається вам оптимальною?
lab.checkpoint("softmax temperature", 0.3)

Відповідь правильна ✅
Окей, добре


0.3

## Top-k sampling

In [51]:
def top_k_sampling(probs, k):
    topk = probs.topk(k)
    index = torch.multinomial(topk.values, num_samples=1)[0]
    print(f"Top {k} words take {topk.values.sum():%} probability mass")
    return topk.indices[index]

In [52]:
k = 15
sample_fn = lambda probs: top_k_sampling(probs, k=k)
for i in range(1, 6):
    result = generate(model, tokenizer, sample_fn, max_len=20)
    print(f"#{i}: {result}")
    print()

Top 15 words take 24.650967% probability mass
Top 15 words take 52.455300% probability mass
Top 15 words take 78.411824% probability mass
Top 15 words take 72.466278% probability mass
Top 15 words take 97.829008% probability mass
Top 15 words take 64.030439% probability mass
Top 15 words take 60.525846% probability mass
Top 15 words take 68.046075% probability mass
Top 15 words take 94.212645% probability mass
Top 15 words take 65.382004% probability mass
Top 15 words take 63.434607% probability mass
Top 15 words take 58.338660% probability mass
Top 15 words take 53.938299% probability mass
Top 15 words take 59.138405% probability mass
Top 15 words take 55.738580% probability mass
Top 15 words take 30.756187% probability mass
Top 15 words take 86.129749% probability mass
Top 15 words take 85.227823% probability mass
Top 15 words take 87.848949% probability mass
#1: <|endoftext|>I know, right? But you know, I'm just gonna get the facts right here,

Top 15 words take 24.650967% probabili

## Nucleus (top-p) sampling

In [55]:
def nucleus_sampling(probs, max_p):
    sorted_probs = probs.sort(descending=True)
    cum_prob = 0.0
    sample_indices = []
    sample_probs = []
    for i in range(0, len(sorted_probs.values)):
        p = sorted_probs.values[i]
        cum_prob += p
        sample_probs.append(p)
        sample_indices.append(sorted_probs.indices[i])
        if max_p <= cum_prob:         # <---------- Implement this
            break

    sample_probs = torch.tensor(sample_probs)
    sample_probs = sample_probs.div(sample_probs.sum())
    index = torch.multinomial(sample_probs, num_samples=1)[0]
    return sample_indices[index]

In [56]:
for max_p in (0.0, 0.1, 0.3, 0.5, 0.6, 0.8, 1.0):
    sample_fn = lambda probs: nucleus_sampling(probs, max_p=max_p)
    result = generate(model, tokenizer, sample_fn, max_len=20)
    print(f"#{max_p}: {result}")
    print()

#0.0: <|endoftext|>
The first time I saw the new version of the game, I was so excited. I

#0.1: <|endoftext|>
The "Big Three" of the Internet are the Internet, the media, and the government

#0.3: <|endoftext|>
In this video, we take a look at the life of the original Titan, the first

#0.5: <|endoftext|>- Introduction -

What's new in the latest version of Java 7?

-

#0.6: <|endoftext|>Dr. Janna and Dr. Mario, who co-founded an organization that aims to bring

#0.8: <|endoftext|> which actually resolves the problem of server data corrupted by uninitialized variable

This proposal proposes using

#1.0: <|endoftext|>_followdetails_simple.cpp:

include "here.h"

GF



## Start from prompt

До цього моменту ми генерували текст з нуля. Однак значно кориснішим є задача генерації тексту від певного префікса або "підказки" -- в англійській мові це називається "prompt".

Prompt дозволить нам контролювати тематику згенерованого тексту або, як ми побачимо на іншій лекції, допоможе моделі виконувати різноманітні завдання.

In [57]:
@torch.no_grad()
def generate(model, tokenizer, sample_fn, prompt, max_len=50, bpe_decode=True):

    result = tokenizer.encode(prompt)     # <---- цей рядок додано

    while len(result) < max_len:

        # Передбачення ймовірностней наступного токена
        input_ = torch.LongTensor([result])
        output = model(input_)
        log_probs = torch.log_softmax(output.logits[0, -1], dim=-1)

        # Обираємо токен, що має найбільшу ймовірність
        token_index = sample_fn(log_probs.exp())

        # Зупиняємося, якщо досягнули кінця
        if token_index == tokenizer.eos_token_id:
            break

        # Додаємо обраний токен в згенерований текст
        result.append(token_index.item())

    if bpe_decode:
        tokens = tokenizer.convert_ids_to_tokens(result)
        result = tokenizer.convert_tokens_to_string(tokens)

    return result

In [62]:
max_p = 0.9
sample_fn = lambda probs: nucleus_sampling(probs, max_p=max_p)
for i in range(1, 5):
    result = generate(model, tokenizer, sample_fn, prompt="Happiness is ")
    print(f"#{i}: {result}")
    print()


# Спробуйте змінити prompt на щось інше

#1: Happiness is _______

Like the Left in its many popularizing references to their madness, "those who practice unhealthy behaviors" are "creative" and need a work ethic. _______ _______ is the Enemy _______

What

#2: Happiness is  embarrassing and off-putting as  it is. But  Prudent Man " My blood... I feel  impossible for my am I rich? " that sounds somewhat interesting, and might in

#3: Happiness is vernacular, but not as widely as it's been because of the next few years. As the people learn and better their understanding of the society, they begin to understand other cultures much more, and it would take more time for

#4: Happiness is ˜¬˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜˜



# Zero-shot класифікація

Ми будемо досліджувати цю тему на окремій лекції. Але спробуємо застосувати мовну модель для задачі класифікації вже зараз. Хоча наша модель надто маленька й слабка для серйозного використання в zero-shot.


In [63]:
generate(model, tokenizer, greedy_choice, prompt="This film is awful! Actor play is terrible. The plot is dull. I would rate it as a").splitlines()[0]


'This film is awful! Actor play is terrible. The plot is dull. I would rate it as a B+.'

In [64]:
generate(model, tokenizer, greedy_choice, prompt="This film is amazing! The time flies by when you watch it. Definitely recommend! On the scale of 1 to 5, I would rate it as a").splitlines()[0]


'This film is amazing! The time flies by when you watch it. Definitely recommend! On the scale of 1 to 5, I would rate it as a 5.'

In [65]:
lab.answer("Готово!")

Відповідь правильна ✅
Ця робота не має однієї правильної відповіді. Вважаємо лабу пройденою :) Формочка: https://tally.so/r/mZ81ve


'Готово!'

# Real world

У цій роботі ми імплементували кілька методів декодінгу. Але, звичайно, все вже зроблено за нас.

Подивіться на параметри функції [generate()](https://huggingface.co/docs/transformers/v4.21.1/en/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate) з бібліотеки transformers. Багато з них мають виглядати знайомими.

Повний приклад використання:

In [66]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

prompt = "I hope that in that practical class on text generation you"

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(**inputs, do_sample=True, top_p=0.8, max_length=50)

tokenizer.batch_decode(outputs, skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['I hope that in that practical class on text generation you will not be forced to do what many would do if you could. I am also proud of my new book, The Great American Novel, on how to tell a story of love, romance,']

# The End


In [67]:
lab.answer("Готово!")

Відповідь правильна ✅
Ця робота не має однієї правильної відповіді. Вважаємо лабу пройденою :) Формочка: https://tally.so/r/mZ81ve


'Готово!'